## Library Descriptions

- **patchright**:  
    Used for patching and managing updates for certain libraries or configurations.

- **lxml**:  
    A powerful library for parsing and working with XML and HTML documents, often used in web scraping.

- **nest_asyncio**:  
    Allows running asynchronous event loops within Jupyter Notebooks, resolving conflicts caused by its built-in event loop.

- **playwright**:  
    A library for browser automation, used for scraping or testing web applications.

- **duckduckgo-search**:  
    Provides programmatic access to DuckDuckGo search results for retrieving web-based information.

- **gradio**:  
    A framework for building user-friendly web-based interfaces, commonly used for showcasing AI applications.

In [ ]:

from typing import TypedDict, Dict, List ,Any
from langgraph.graph import StateGraph, END, START, MessagesState
from IPython.display import display, Image
from langchain_core.runnables.graph import MermaidDrawMethod
from langchain.tools import DuckDuckGoSearchResults
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage 

from langsmith import Client
from langsmith import traceable


import asyncio
import time

import os
from dotenv import load_dotenv

# For scraping
from patchright.async_api import async_playwright
from lxml import html
from abc import ABC, abstractmethod
import re

# This import is required only for jupyter notebooks, since they have their own eventloop
import nest_asyncio
nest_asyncio.apply()


#Web Scraping and Interface

### Web Scraping and Interface Definitions

In [ ]:
async def scroll_to_bottom(page, scroll_delay=0.1):
    """
    Scroll to the bottom of the page iteratively, with delays to ensure dynamic content is fully loaded.
    
    Args:
        page: The Playwright page instance.
        scroll_delay: Delay in seconds between scrolls to allow content loading.
    """
    
    print("Scrolling through the page...")
    
    scroll_size = 2160

    next_scroll = scroll_size
    for i in range(3):
        # Scroll 500 pixels at a time
        await page.evaluate(f"window.scrollTo(0, {next_scroll})")

        next_scroll += scroll_size

        # Wait for content to load
        await asyncio.sleep(scroll_delay)
        
    print("Finished scrolling through the page.")

async def block_unnecessary_resources(route):
    if route.request.resource_type in ["image"]:
        await route.abort()
    else:
        await route.continue_()
        
class WebsiteInterface(ABC):
    def __init__(self):
        self.base_url = ""
        
    @abstractmethod
    async def crawl(self) -> List[Dict[str, str]]:
        """
        Abstract method to crawl the website and extract listings.
        Must be implemented by subclasses.
        """
        pass

    @abstractmethod
    def get_filters_info(self) -> str:
        """
        Abstract method to return a prompt for the LLM describing the filters and expected output format.
        Must be implemented by subclasses.
        """
        pass

    @abstractmethod
    def set_filters_from_llm_response(self, llm_response: str):
        """
        Abstract method to process the LLM's response and set the URL with appropriate filters.
        Must be implemented by subclasses.
        """
        pass

class AutotraderInterface(WebsiteInterface):
    def __init__(self):
        self.base_url = "https://www.autotrader.com/cars-for-sale/all-cars"
        # https://www.autotrader.com/cars-for-sale/all-cars/floral-park-ny?endYear=2022&makeCode=BMW&makeCode=FORD&newSearch=true&startYear=2012&zip=11001
        
    async def crawl(self) -> List[Dict[str, str]]:
        listings = []
        
        url = self.url

        playwright = await async_playwright().start()

        # Launch browser in headless mode
        browser = await playwright.chromium.launch(headless=True,
                                                    args=[
                                                            "--no-sandbox",
                                                            "--disable-setuid-sandbox",
                                                            "--disable-dev-shm-usage",
                                                            "--disable-extensions",
                                                            "--disable-gpu"
                                                    ]
                                                    )
        
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
            viewport={"width": 1920, "height": 1080},
            # no_viewport=True
            locale="en-US",
            timezone_id="America/New_York",
            # java_script_enabled=False,
        )


        print("Opening browser page")

        page = await context.new_page()
        
        await page.route("**/*", block_unnecessary_resources)

        print("Loading page")
        
        await page.goto(url, wait_until="domcontentloaded")

        print("Page partially loaded. Starting to scroll.")
        
        # Scroll to the bottom of the page
        await scroll_to_bottom(page)
        
        page_content = await page.content()
        
        # Parse HTML using lxml
        tree = html.fromstring(page_content)

        # XPath to select each car listing container
        listings_elements = tree.xpath('//div[@data-cmp="inventoryListing"]')

        listings = []

        for listing in listings_elements:
            car_data = {}
            # Extract car details
            car_data['title'] = listing.xpath('.//h2[@data-cmp="subheading"]/text()')
            car_data['mileage'] = listing.xpath('.//div[@data-cmp="mileageSpecification"]/text()')
            car_data['price'] = listing.xpath('.//div[@data-cmp="firstPrice"]/text()')
            car_data['dealer'] = listing.xpath('.//div[@class="text-subdued"]/text()')
            car_data['phone'] = listing.xpath('.//span[@data-cmp="phoneNumber"]/text()')
            car_data['url'] = listing.xpath('.//a[@data-cmp="link"]/@href')
            car_data['image'] = listing.xpath('.//img[@data-cmp="inventoryImage"]/@src')
            
            # Clean up extracted data
            car_data = {key: (val[0].strip() if val else None) for key, val in car_data.items()}
            
            car_data['url'] = car_data['url'].split('?')[0]
            
            # Add domain to the URL. Extract domain from the base URL without the path
            car_data['url'] = re.sub(r'^(https?://[^/]+).*$', r'\1', self.base_url) + car_data['url']
            
            # Set the ID of the listing as the ID of the WebsiteInterface and the car number from URL
            car_data = { "id": f"{self.__class__.__name__}_{car_data['url'].split('/')[-1]}" } | car_data
            
            listings.append(car_data)
            
        if __name__ == "__main__":
            print("Found the following car listings:")
            # Display the extracted data
            for car in listings:
                print(car)

        print("Found", len(listings), "listings")

        await browser.close()
        
        return listings
    
    async def crawl_listing(self, listing_url) -> List[Dict[str, str]]:
        listing_info = ""
        
        url = listing_url

        playwright = await async_playwright().start()

        # Launch browser in headless mode
        browser = await playwright.chromium.launch(headless=True,
                                                    args=[
                                                            "--no-sandbox",
                                                            "--disable-setuid-sandbox",
                                                            "--disable-dev-shm-usage",
                                                            "--disable-extensions",
                                                            "--disable-gpu"
                                                    ]
                                                    )
        
        context = await browser.new_context(
            user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36',
            viewport={"width": 1920, "height": 1080},
            # no_viewport=True
            locale="en-US",
            timezone_id="America/New_York",
            # java_script_enabled=False,
        )


        print("Opening browser page")

        page = await context.new_page()
        
        await page.route("**/*", block_unnecessary_resources)

        print("Loading page")
        
        await page.goto(url, wait_until="domcontentloaded")

        print("Page partially loaded. Starting to scroll.")

        # Scroll to the bottom of the page
        await scroll_to_bottom(page)

        # Get full HTML
        page_content = await page.content()

        # Parse HTML using lxml to extract all the text
        tree = html.fromstring(page_content)
        listing_info = tree.xpath("//div[contains(@class, 'container') and contains(@class, 'margin-top-5')]/div[contains(@class, 'row')]//text()")
        listing_info = "\t".join(listing_info).strip()

        # Seller information should already be included in the listing information
        # seller_info = tree.xpath("//div[@id='sellerComments']//text()")
        # listing_info = listing_info + seller_info
            
        if __name__ == "__main__":
            print("Found the following information:")
            # Print the extracted text
            print(listing_info)

        await browser.close()
        
        return listing_info
    
    def get_filters_info(self) -> str:
        """
        Return a prompt for the LLM describing the filters and expected output format.
        """
        return f"""
        You are a helpful assistant that translates user requirements into a URL with query parameters.

        The base URL is: {self.base_url}
        Filters:
        - zip: User's zip code (integer).
        - searchRadius: Search radius in miles (integer, e.g., 75, 100, 200).
        - startYear: Minimum year of the car (integer).
        - endYear: Maximum year of the car (integer).
        - makeCode: Car manufacturer code (string, can appear multiple times, e.g., "BMW", "FORD").
        - listingType: Type of listing (one of "NEW", "USED", "CERTIFIED", "3P_CERT").
        - mileage: Maximum mileage of the car (integer).
        - driveGroup: Type of drive (one of "AWD4WD", "FWD", "RWD").
        - extColorSimple: External color of the car (e.g., "BLACK", "WHITE", "RED", "GRAY").
        - intColorSimple: Internal color of the car (e.g., "BEIGE", "BLACK", "BLUE").
        - mpgRange: Fuel efficiency in miles per gallon (e.g., "30-MPG").
        - fuelTypeGroup: Type of fuel (one of "GSL", "DSL", "HYB", "ELE", "PIH").
        - bodyStyleSubtypeCode: Type of body style (e.g., "FULLSIZE_CREW", "COMPACT_EXTEND").
        - truckBedLength: Truck bed length (e.g., "SHORT", "EXTRA SHORT", "UNSPECIFIED").
        - vehicleStyleCode: Vehicle style (e.g., "CONVERT", "WAGON", "HATCH", "SUVCROSS").
        - dealType: Type of deal (e.g., "goodprice", "greatprice").
        - doorCode: Number of doors (e.g., "2", "3", "4").
        - engineDisplacement: Engine size range in liters (e.g., "1.0-1.9", "2.0-2.9").
        - featureCode: Specific features of the car (e.g., "1062" for heated seats, "1327" for navigation).
        - transmissionCode: Transmission type (e.g., "AUT" for automatic, "MAN" for manual).
        - vehicleHistoryType: Vehicle history (e.g., "NO_ACCIDENTS", "ONE_OWNER", "CLEAN_TITLE").
        - newSearch: Boolean to indicate a new search (e.g., "true").
        - sortBy: Sorting option for the results (optional). 
            Options:
            - "relevance" (default): Sort by relevance.
            - "derivedpriceASC": Sort by price, lowest to highest.
            - "derivedpriceDESC": Sort by price, highest to lowest.
            - "distanceASC": Sort by distance, closest to farthest.
            - "datelistedASC": Sort by date, oldest first.
            - "datelistedDESC": Sort by date, newest first.
            - "mileageASC": Sort by mileage, lowest to highest.
            - "mileageDESC": Sort by mileage, highest to lowest.
            - "yearASC": Sort by year, oldest to newest.
            - "yearDESC": Sort by year, newest to oldest.
        
        Special filters:
        - price: Price is embedded in the path of the URL, e.g., "/cars-over-45000" or "/cars-between-10000-and-20000".

        Example Output:
        A complete URL with query parameters, e.g.,:
        "{self.base_url}/cars-between-10000-and-20000?zip=10001&startYear=2010&endYear=2020&makeCode=BMW&makeCode=FORD&listingType=USED&mileage=50000&fuelTypeGroup=GSL&intColorSimple=BLACK&vehicleHistoryType=NO_ACCIDENTS"

        Based on the user's needs, format the response as only the complete URL (no extra explanations). The URL is an example, don't include filters if they are not needed by the user.
        """
        
    def set_filters_from_llm_response(self, llm_response: str):
        """
        Process the LLM's response and set the URL with the provided parameters.
        """
        # Validate and set the URL from LLM's response
        if llm_response.startswith(self.base_url):
            self.url = llm_response.strip()
        else:
            raise ValueError("Invalid URL format provided by LLM response: " + llm_response)

In [ ]:
from langchain_ollama import ChatOllama

GPT = ChatOllama(model="mistral", temperature=0, max_retries=2)

### Define the State Class

This cell defines the `State` class, which inherits from `MessagesState`, to represent the current state of the car-buying process. It includes:

- **user_needs**: Stores the user s requirements for the car (e.g., budget, features).
- **web_interfaces**: A list of web scraper interfaces (e.g., AutoTrader) to fetch car listings.
- **listings**: A collection of car listings retrieved from the web platforms.
- **selected_listing**: The specific car listing chosen by the user for further exploration.
- **additional_info**: Additional information about the selected car (e.g., common issues, reliability).
- **next_node**: The next action or state transition in the workflow.

In [ ]:
class State(MessagesState):
    user_needs: Dict[str, Any] = {}
    web_interfaces: List[WebsiteInterface] = []
    listings: List[Dict[str, Any]] = []
    selected_listing: Dict[str, Any] = {}
    additional_info: Dict[str, Any] = {}
    next_node: str = ""

In [ ]:
def get_user_input(*args, **kwargs):
    """Get user input."""
    return input(*args, **kwargs)

def show_assistant_output(*args, **kwargs):
    """Show the output of the LLM."""
    print(*args, **kwargs)

In [ ]:
from typing import TypedDict
from enum import Enum
from pydantic import BaseModel
import json

class NextStep(Enum):
    ASK_USER_NEEDS = "ask_user_needs"
    BUILD_FILTERS = "build_filters"
    IRRELEVANT = "irrelevant"

class UserNeeds(BaseModel):
    user_needs: str
    next_step: NextStep

USER_NEEDS_GPT = ChatOllama(model="mistral", response_format=UserNeeds)

def ask_user_needs(state: State) -> State:
    """Ask user initial questions to define their needs for the car."""
    messages = state.get("messages", [])    
    if len(messages) == 0:
        system_message = "You are a car buying assistant. Your goal is to help the user find a car that meets their needs. Start by introducing yourself and asking about their requirements, such as intended usage (e.g., commuting, family trips), budget, size preferences, and any specific constraints or features they value. Use their responses to guide them toward the best options."
    else:
        system_message = "Ask the user for any additional information that can help narrow down the search. If he asked any questions before, answer them before asking for more information. When answering, make sure to provide clear and concise information, with relevant examples."
        
    existing_needs = state.get("user_needs", "")
    if existing_needs:
        system_message += f" Here's what we know about the needs of the user so far:\n\n{existing_needs}"

    messages.append(SystemMessage(content=system_message))

    # Get message from the LLM
    response = GPT.invoke(messages).content
    messages += [AIMessage(response)]
    show_assistant_output(f"\033[92m{messages[-1].content}\033[0m", flush=True)
    
    messages += [HumanMessage(get_user_input(response))]
    print(f"\033[94m{messages[-1].content}\033[0m", flush=True)
    
    summarization_messages = messages.copy()
    
    summarization_messages += [
        SystemMessage(
            "Summarize the user's car-buying needs in clear and concise bullet points based on their input and any prior knowledge.\n"
            "Provide the next step, such as asking for more details or answer questions under ask_user_needs or going forward to build_filter:\n"
            "- Use 'ask_user_needs' if you need more information or if the user asked a question.\n"
            "- Use 'build_filters' if you have enough details to search for cars online.\n"
            "If the user's query is irrelevant to the matter at hand (buying a car), respond 'irrelevant'."
        )
    ]
    
    # Get the structured response directly as a Pydantic model
    try:
        response_model = USER_NEEDS_GPT.invoke(summarization_messages)
        response = response_model.content  # This should be a UserNeeds instance
    except Exception as e:
        # Fallback in case of error
        print(f"Error getting structured response: {e}")
        response = UserNeeds(
            user_needs="Could not parse user needs from conversation",
            next_step=NextStep.ASK_USER_NEEDS
        )

    state["user_needs"] = response.user_needs
    
    messages += [AIMessage("I have summarized your car-buying needs as follows:\n" + state["user_needs"])]
    
    show_assistant_output(f"\033[92m{messages[-1].content}\033[0m")
    
    state["next_node"] = response.next_step.value  # Get the string value from the Enum
        
    print(f"\nNext node: {state['next_node']}", flush=True)

    return state

In [ ]:
def build_filters(state: State) -> State:
    """Build and refine search filters based on user needs."""

    show_assistant_output("Building filters based on user needs...")
    
    for interface in state["web_interfaces"]:
        filters_info = interface.get_filters_info()
        
        # TODO: Check if this website is useful to the user based on the filters
        # If not continue to the next interface
        
        # If the website is useful, use LLM to setup the filters based on user needs
        
        # Define system instructions with filters information
        system_message = SystemMessage(filters_info + "\n\n" + "User needs:\n" + state["user_needs"])

        # Use the LLM to process the user's needs and set the filters
        try:
            result = GPT.invoke([system_message])
            llm_response = result.content.strip()

            # Validate and set the filters for the interface
            interface.set_filters_from_llm_response(llm_response)
            show_assistant_output(f"\nSuccessfully set filters for: {interface.__class__.__name__}")
            show_assistant_output(f"Updated URL: {interface.url}")
        except ValueError as e:
            show_assistant_output(f"Failed to set filters for {interface.base_url}: {e}")
        except Exception as e:
            show_assistant_output(f"An error occurred while processing filters for {interface.base_url}: {e}")
    
    return

In [ ]:
async def fetch_listings_from_sources(web_interfaces: List[WebsiteInterface]) -> List[Dict[str, str]]:
    """Simulate retrieval of car listings from Autotrader.com based on filters.
    
    Args:
        filters (dict): Dictionary containing search filters (e.g., budget, fuel type).
        
    Returns:
        list: A list of dictionaries, each representing a car listing.
    """
    listings = []
    for interface in web_interfaces:
        listings += await interface.crawl()
        
    return listings

In [ ]:
from typing import Literal, Optional

class UserResponse(BaseModel):
    action: Literal['select_listing', 'refine_search', 'end_conversation']
    listing_id: Optional[str]

CLASSIFIER_GPT = ChatOllama(model="mistral", response_format=UserResponse)

def search_listings(state: State) -> State:
    """Search for cars on LaCentrale and mobile.de based on filters."""
    """Display the first listings for the user to view."""
    """Synchronous wrapper for search_listings."""

    state["messages"] += [SystemMessage("Searching for listings based on user needs, this may take time...")]
    show_assistant_output(state["messages"][-1].content)

    async def _search_listings():
        return await fetch_listings_from_sources(state["web_interfaces"])
    
    listings = asyncio.run(_search_listings())
    state["listings"] = listings
    
    show_assistant_output(f"Successfully fetched {len(listings)} listings from the sources.")
    
    AI_message = ""
    
    # Display the first few listings for the user to view
    AI_message += "Here are recent listings that match your requirements:\n"
    for i, listing in enumerate(state["listings"][:5], 1):
        AI_message += f"{i}.\n"
        for key, value in listing.items():
            formatted_key = key.replace("_", " ").capitalize()
            if formatted_key == "Image" and value:
                AI_message += f"   {formatted_key}: ![Example Image]({value})\n"
            else:
                AI_message += f"   {formatted_key}: {value}\n"
        AI_message += "\n"  # Add an extra line for readability
    
    user_prompt = "Would you like to view more details about a specific listing, or refine your search (Write END to finish this conversation) ?"
    AI_message += user_prompt
        
    state["messages"].append(AIMessage(AI_message))
    show_assistant_output(f"\033[92m{state['messages'][-1].content}\033[0m")
    state["messages"].append(HumanMessage(get_user_input(user_prompt)))
    print(f"\033[94m{state['messages'][-1].content}\033[0m")
       
    response = json.loads(CLASSIFIER_GPT.invoke(state["messages"]).content)

    if response["action"] == "select_listing":
        state["next_node"] = "fetch_additional_info"
        selected_listing_id = response["listing_id"]
        for i, listing in enumerate(state["listings"][:5], 1):
            if selected_listing_id in listing["id"]:
                state["selected_listing"] = listing
                break
    elif response["action"] == "refine_search":
        state["next_node"] = "ask_user_needs"
    else:
        state["next_node"] = END
        
    return state

In [ ]:
from langchain_community.tools import DuckDuckGoSearchResults

duckduckgo_search = DuckDuckGoSearchResults(max_results=3)

def fetch_additional_info(state: State) -> State:
    """Fetch more details about the selected car listing."""
    listing = state["selected_listing"]

    # Crawl the car listing page to get more details about the car for sale and the seller

    async def _crawl_car_listing():
        for interface in state["web_interfaces"]:
            if listing["id"].split("_")[0].lower() in interface.__class__.__name__.lower():
                return await interface.crawl_listing(listing["url"])
    
    info_car_for_sale = asyncio.run(_crawl_car_listing())

    # Call the LLM to summarize the information about the car for sale into a concise paragraph
    prompt = SystemMessage(
        f"Summarize all the relevant information about the selected car for sale into a paragraph: {listing['title']}\n\n"
        f"Here is the raw information about the car for sale:\n\n{info_car_for_sale}"
        f"Format the summary clearly and concisely, with line breaks between sections."
    )

    car_info_summary = GPT.invoke([prompt]).content

    show_assistant_output("\033[92mHere are more details about the car for sale:\n\033[0m", flush=True)

    show_assistant_output("\033[92m" + car_info_summary + "\n\n\033[0m", flush=True)

    state["messages"] += [prompt, AIMessage(car_info_summary)]

    # Search for common issues and reliability of the car on DuckDuckGo
    car_name = listing["title"]

    queries = [f"{car_name} common issues", f"{car_name} problem", f"{car_name} reliability"]
    context = ""
    for query in queries:
        search_results = duckduckgo_search.invoke(query)
        formatted_results = f"QUERY: {query}\n\n{search_results}\n-------------------\n"
        context += formatted_results

    prompt = SystemMessage(
        f"Provide additional information about this car: {listing['title']}, "
        f"including engine specifications, common issues with this model, and market value."
        f"Here is additioanl context to help you provide the information:\n\n{context}"
        f"Here are the user needs, give some insights about the car based on the user needs:\n\n{state['user_needs']}"
    )
    
    result = GPT.invoke([prompt])
    
    listing["additional_info"] = result.content
    
    show_assistant_output(f"\033[92mHere is additional information about the model in general, coming from Internet:\n{listing['additional_info']}\n\033[0m")
    
    user_prompt = "Would you like to view more details about another listing, or refine your search (Write END to finish this conversation) ?"
    state["messages"] += [SystemMessage(user_prompt)]
    state["messages"] += [HumanMessage(get_user_input(user_prompt))]
    print(f"\033[94m{state['messages'][-1].content}\033[0m", flush=True)
    
    response = json.loads(CLASSIFIER_GPT.invoke(state["messages"]).content)

    if response["action"] == "select_listing":
        state["next_node"] = "fetch_additional_info"
        selected_listing_id = response["listing_id"]
        for i, listing in enumerate(state["listings"][:5], 1):
            if selected_listing_id in listing["id"]:
                state["selected_listing"] = listing
                break
    elif response["action"] == "refine_search":
        state["next_node"] = "ask_user_needs"
    else:
        state["next_node"] = END
    
    return state

In [ ]:
# Initialize the StateGraph
workflow = StateGraph(State)

# Define the nodes in the graph
workflow.add_node("ask_user_needs", ask_user_needs)
workflow.add_node("build_filters", build_filters)
workflow.add_node("search_listings", search_listings)
workflow.add_node("fetch_additional_info", fetch_additional_info)
workflow.add_node("irrelevant", lambda state: state)

# Define edges
workflow.add_conditional_edges("ask_user_needs", lambda state: state["next_node"], ["build_filters", "ask_user_needs", "irrelevant"])
workflow.add_edge("build_filters", "search_listings")
workflow.add_conditional_edges("search_listings", lambda state: state["next_node"], ["fetch_additional_info", "ask_user_needs", END])
workflow.add_edge("irrelevant", END)

# Set the entry and exit points
workflow.set_entry_point("ask_user_needs")
workflow.add_conditional_edges("fetch_additional_info", lambda state: state["next_node"], ["ask_user_needs", "fetch_additional_info", END])


# Compile the workflow
app = workflow.compile()

In [ ]:
display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)

In [ ]:
def run_car_buyer_agent():
    """Run the car-buying assistant with LangGraph."""
        
    messages = []
    
    initial_state = State(
        user_needs={}, 
        web_interfaces=[AutotraderInterface()], 
        listings=[],
        selected_listing={}, 
        additional_info={},
        next_node="",
        messages=messages
    )
    result = app.invoke(initial_state)
    return result

In [ ]:
USE_GRADIO = True

if not USE_GRADIO:
    def get_user_input(*args, **kwargs):
        """Get user input."""
        return input(*args, **kwargs)

    def show_assistant_output(*args, **kwargs):
        """Show the output of the LLM."""
        print(*args, **kwargs)

    # Execute the agent
    car_buyer_result = run_car_buyer_agent()

    # Print result for debugging purposes
    print("Car Buyer Result:", car_buyer_result)

    # Display summary of the final recommendation
    if "selected_listing" in car_buyer_result:
        listing = car_buyer_result["selected_listing"]
        print(f"\nFinal Recommendation:\n{listing['title']} - {listing['price']} - {listing['mileage']} km")
        print("Additional Information:")
        for key, value in car_buyer_result["additional_info"].items():
            print(f"{key}: {value}")
    else:
        print("No car listing selected.")

In [ ]:
import threading
import queue
import gradio as gr
from gradio import ChatMessage
import time
import re

waiting_for_input = False # Flag to indicate if the agent is waiting for user input

class InputQueue:
    """A custom input queue that mimics stdin behavior."""
    def __init__(self):
        self.queue = queue.Queue()

    def readline(self):
        """Mimic the readline behavior of stdin."""
        try:
            r = self.queue.get(block=True)  # Wait until input is available
            return r
        except queue.Empty:
            return ""

    def write(self, message):
        """Handle writes if needed (for debugging)."""
        pass

    def flush(self):
        """No-op for compatibility."""
        pass

    def put(self, message):
        """Put a message into the queue."""
        self.queue.put(message)

# A thread-safe queue for communication
output_queue = queue.Queue()
# Replace sys.stdin with the custom InputQueue
input_queue = queue.Queue()

def get_user_input(*args, **kwargs):
    """Get user input."""
    global waiting_for_input
    print("Waiting for user input...")
    waiting_for_input = True
    r = input_queue.get()
    waiting_for_input = False
    print(f"Received user input")
    return r

def show_assistant_output(*args, **kwargs):
    """Show the output of the LLM."""
    
    result = " ".join(args) + kwargs.get("end", "\n")
    
    # Replace any Color Codes with Regex
    result = re.sub(r'\033\[\d+m', '', result)
    # result = result.replace("\033[92m", "").replace("\033[0m", "").replace("\033[94m", "")
    
    output_queue.put(result)

# Gradio UI Functionality
def interact_with_agent(user_message, history, discard_user_input=False):
    """Send user message to the bot and handle the response."""
    
    global waiting_for_input
    
    if not discard_user_input:
        input_queue.put(user_message + "\n")  # Send user input to LangGraph
        
    partial_message = ""

    # Fetch and yield bot responses incrementally
    while True:
        try:
            message = output_queue.get(timeout=0.1)  # Wait for bot output
            if message:
                    
                partial_message += message
                yield partial_message
        except queue.Empty:
            is_end = waiting_for_input
            if is_end:
                break
            time.sleep(0.1)
            
def get_initial_message():
    """Run the agent and capture the initial message."""
    # Simulate an initial empty input to get the initial message
    initial_message = ""
    
    for message in interact_with_agent("", [], True):  # Consume the generator to get the full initial message
        initial_message = message  # Keep updating until the generator finishes

    return initial_message

initial_message_content = ""

def run_langgraph_agent():
    """Run the LangGraph agent and redirect its stdout."""
    global initial_message_content
    run_car_buyer_agent()  # Start the LangGraph workflow


if USE_GRADIO:
    # Run the agent in a separate thread
    agent_thread = threading.Thread(target=run_langgraph_agent, daemon=True)
    agent_thread.start()

    initial_message_content = get_initial_message()

    initial_messages = [{"role": "assistant", "content": initial_message_content}]

    chat = gr.ChatInterface(interact_with_agent,
                    chatbot=gr.Chatbot(label="Car Buyer Chatbot", autoscroll=True, scale=1, value=initial_messages, type="messages", height=200),
                    type="messages").launch()